In [1]:
import numpy as np
import sympy as sp
import pickle
from IPython.display import HTML
import ipywidgets as widgets
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
mpl.rcParams['legend.fontsize'] = 10
import pandas as pd
import itertools
pd.set_option('display.max_colwidth', None)
from sympy.plotting import plot 
from IPython.display import Image

In [35]:
# Render to Latex function 
def RTL(e):
    latex_rendering = []

    for i in range(len(e)):
        latex_rendering.append("$" + sp.latex(e[i]) + "$ &nbsp;&nbsp;")
    
    return(HTML("".join(latex_rendering[0:])))


# Plot vectors with customisation
def PlotVectors(vectors = [], plotPointsAsShape = False, plotFromStandardPosition = False):

    plt.scatter(0, 0)
    plt.grid()
    
    xValues = [vectors[i][0] for i in range(len(vectors))]
    yValues = [vectors[i][1] for i in range(len(vectors))]
    
    plt.scatter(xValues, yValues)
    
    
    if plotPointsAsShape:
        xValues.append(xValues[0])
        yValues.append(yValues[0])

        plt.plot(xValues, yValues)
    
    if plotFromStandardPosition:
        for i in range(len(xValues)):
            plt.plot([0, xValues[i]], [0,  yValues[i]])

            
# Apply transformation to multiple vectors
def ApplyTransformationViaScaledIdentityMatrix(scaledIdentityMatrix, vectors):
    transformedVectors = []
    for i in vectors: 
        t = scaledIdentityMatrix * i
        transformedVectors.append(t)
        
    return(transformedVectors)


def ConvertSolutionToVectorForm(solutionList):
    
    SolutionsInVectorForm = []
    NumberOfRows = len(solutionList)
    FreeSymbols = list(solutionList.free_symbols)
    SubsDict = {}
    for i in FreeSymbols:
        SubsDict[i] = 0
    Constants = np.array(solutionList.subs(SubsDict))
    SolutionsInVectorForm.append(Constants)

    for i in range(len(SubsDict)): 

        TempDict = SubsDict.copy()
        TempDict[list(SubsDict.keys())[i]] = 1
        Variable =  (np.array(solutionList.subs(TempDict)) - Constants) * list(SubsDict.keys())[i]
        SolutionsInVectorForm.append(Variable)
      

  
    return(sp.Matrix(SolutionsInVectorForm).T)
   
def MatrixRowsToAtomic(Row):
    
    NumberOfRows = Row.shape[0]
    
    
    ArgList = np.array([])
    
    for i in range(len(Row)):
        if " " in str(Row[i]):
            Args = np.array(Row[i].args)
            
            ArgList = np.append(ArgList, Args)
            print('SIMP', ArgList)
        else: 
            ArgList = np.append(ArgList, Row[i])
            print('NO SIMP', ArgList)
    SortedArgList = list(ArgList)
    
   # arr_3d = np. reshape(SortedArgList, (2, 2, 3))
    
    return(SortedArgList)
    
    

In [11]:
#EX 2.11
# Let E1 be a augmented matrix
E1 = sp.Matrix([[1, -1, -1 ,2, 1], [2, -2, -1 ,3, 3], [-1, 1, -1 ,0, -3]])
E1

Matrix([
[ 1, -1, -1, 2,  1],
[ 2, -2, -1, 3,  3],
[-1,  1, -1, 0, -3]])

In [12]:
# Create some variables 
w, x, y, z, s, t = sp.symbols('w, x, y, z, s, t')
# Let E2 be a solution to augmented matrix
E2 = sp.linsolve(E1, (w, x, y, z ))
E2

FiniteSet((x - z + 2, x, z + 1, z))

In [13]:
# Note tht this answer can be converted into vector form

In [14]:
# E3 = E2.subs({x: s, z: t})
E4 = list(E2)[0]
E5 = ConvertSolutionToVectorForm(E4)
E5

Matrix([
[2, -z, x],
[0,  0, x],
[1,  z, 0],
[0,  z, 0]])

In [15]:
# Find the rank of the above matrix
E1.rref()[0]

Matrix([
[1, -1, 0,  1, 2],
[0,  0, 1, -1, 1],
[0,  0, 0,  0, 0]])

In [16]:
# Note rank is number of of non-zero rows
E1.rank()

2

In [17]:
#EX 2.13

E1 = sp.Matrix([[1, -1, 2, 3],
               [1, 2, -1, -3],
               [0, 2, -2, 1]])
E1

Matrix([
[1, -1,  2,  3],
[1,  2, -1, -3],
[0,  2, -2,  1]])

In [18]:
# Create some variables 
w, x, y, z, s, t = sp.symbols('w, x, y, z, s, t')
# Let E2 be a solution to augmented matrix
E2 = sp.linsolve(E1, (x, y, z ))
E2

EmptySet

In [19]:
# Note it has no solutions
E1.rref()[0]

Matrix([
[1, 0,  1, 0],
[0, 1, -1, 0],
[0, 0,  0, 1]])

In [20]:
#EX 2.14
E1 = sp.Matrix([[1, 2, -1, 3], 
               [2,3, 1, 1]])

In [21]:
# Create some variables 
w, x, y, z, s, t = sp.symbols('w, x, y, z, s, t')
# Let E2 be a solution to augmented matrix
E2 = sp.linsolve(E1, (x, y, z ))
E2

FiniteSet((-5*z - 7, 3*z + 5, z))

In [22]:
# Now put into vector form
# E3 = E2.subs({x: s, z: t})
E3 = list(E2)[0]
E4 = ConvertSolutionToVectorForm(E3)
E4

Matrix([
[-7, -5*z],
[ 5,  3*z],
[ 0,    z]])

In [23]:
# Check rref
E1.rref()[0]

Matrix([
[1, 0,  5, -7],
[0, 1, -3,  5]])

In [24]:
# EX 2.15
# Determine where the lines P + tU and Q + tV intersect

In [25]:
x, s, t = sp.symbols('x, s, t')

P = sp.Matrix([1, 0, -1])
Q = sp.Matrix([0, 2, 1])
U = sp.Matrix([1, 1, 1])
V = sp.Matrix([3, -1, -1])
RTL([P, Q, U, V])

In [26]:
# Note the lines are independent and so are thier parmas
E1 = P + s * U
E2 = Q + t * V
RTL([E1, E2])

In [46]:
#EX 2.2
E1 = sp.Matrix([[1, 0, 3, -4, 0], 
               [0, 0, 0, 0, 0], 
               [0, 1, 5, 0, 1]])
E1

Matrix([
[1, 0, 3, -4, 0],
[0, 0, 0,  0, 0],
[0, 1, 5,  0, 1]])

In [47]:
E1.echelon_form()

Matrix([
[1, 0, 3, -4, 0],
[0, 1, 5,  0, 1],
[0, 0, 0,  0, 0]])

In [48]:
E1.rref()[0]

Matrix([
[1, 0, 3, -4, 0],
[0, 1, 5,  0, 1],
[0, 0, 0,  0, 0]])

In [49]:
# 3
E1 = sp.Matrix([[0, 1, 3, 0], 
               [0, 0, 0, 1]])

In [50]:
E1

Matrix([
[0, 1, 3, 0],
[0, 0, 0, 1]])

In [51]:
E1.echelon_form()

Matrix([
[0, 1, 3, 0],
[0, 0, 0, 1]])

In [54]:
#2.2-7
E1 = sp.Matrix([[1, 2, 3],
               [1, 0, 0], 
               [0, 1, 1],
               [0, 0, 1]])
E1

Matrix([
[1, 2, 3],
[1, 0, 0],
[0, 1, 1],
[0, 0, 1]])

In [53]:
E1.echelon_form()

Matrix([
[1,  2,  3],
[0, -2, -3],
[0,  0,  1],
[0,  0,  0]])

In [55]:
E1.rref()[0]

Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 1],
[0, 0, 0]])

In [56]:
#2.2-9
E1 = sp.Matrix([[0, 0, 1], 
               [0, 1, 1], 
               [1, 1, 1]])
E1

Matrix([
[0, 0, 1],
[0, 1, 1],
[1, 1, 1]])

In [57]:
E1.echelon_form()

Matrix([
[1, 1, 1],
[0, 1, 1],
[0, 0, 1]])

In [59]:
E1.rref()[0]

Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 1]])

In [60]:
E1 = sp.Matrix([[3, 5], 
               [5, -2],
               [2, 4]])
E1

Matrix([
[3,  5],
[5, -2],
[2,  4]])

In [61]:
E1.echelon_form()

Matrix([
[3,   5],
[0, -31],
[0,   0]])

In [62]:
E1.rref()[0]

Matrix([
[1, 0],
[0, 1],
[0, 0]])

In [63]:
E1 = sp.Matrix([[3, -2, -1], 
               [2, -1, -1],
               [4, -3, -1]])

E1

Matrix([
[3, -2, -1],
[2, -1, -1],
[4, -3, -1]])

In [65]:
E1.rref()[0]

Matrix([
[1, 0, -1],
[0, 1, -1],
[0, 0,  0]])

In [90]:
E1 = sp.Matrix([[1, 2, -4, -4, 5],
               [0, -1, 10, 9, -5],
               [0, 0, 1, 1, -1],
               [0, 0, 0, 0, 24]])
E1

Matrix([
[1,  2, -4, -4,  5],
[0, -1, 10,  9, -5],
[0,  0,  1,  1, -1],
[0,  0,  0,  0, 24]])

In [91]:
#E1.elementary_row_op('n<->m', row1=0, row2 =3)
#   * "n->kn" (row n goes to k*n)
#   * "n<->m" (swap row n and row m)
#   * "n->n+km" (row n goes to row n + k*row m)


In [92]:
E1 = E1.elementary_row_op("n->n+km", row1 = 1, row2=0, k = 1)
E1

Matrix([
[1, 2, -4, -4,  5],
[1, 1,  6,  5,  0],
[0, 0,  1,  1, -1],
[0, 0,  0,  0, 24]])

In [93]:
E1 = E1.elementary_row_op("n<->m", row1=1, row2=2)
E1

Matrix([
[1, 2, -4, -4,  5],
[0, 0,  1,  1, -1],
[1, 1,  6,  5,  0],
[0, 0,  0,  0, 24]])

In [94]:
E1 = E1.elementary_row_op("n->kn", row=1, k = 5)
E1

Matrix([
[1, 2, -4, -4,  5],
[0, 0,  5,  5, -5],
[1, 1,  6,  5,  0],
[0, 0,  0,  0, 24]])

In [95]:
E1 = E1.elementary_row_op('n->kn', row=1, k = 2)
E1

Matrix([
[1, 2, -4, -4,   5],
[0, 0, 10, 10, -10],
[1, 1,  6,  5,   0],
[0, 0,  0,  0,  24]])

In [81]:
E1 = E1.elementary_row_op('n->n+km', row1 = 0, row2=1, k = 4)
E1

Matrix([
[4,  0, 64, 56, -20],
[0, -1, 10,  9,  -5],
[0,  0,  1,  1,  -1],
[0,  0,  0,  0,  24]])

In [97]:
E1 = sp.Matrix([[1, 2], [3, 4]])
E2 = sp.Matrix([[3, -1], [1, 0]])
RTL([E1, E2])

In [98]:
E1 == E2

False

In [99]:
E1.rref()

(Matrix([
 [1, 0],
 [0, 1]]),
 (0, 1))

In [110]:
x, y = sp.symbols('x, y')
E1 = sp.Matrix([[1, 1, 0],
               [1, sp.Rational(801,800), 1]])

In [111]:
E1

Matrix([
[1,       1, 0],
[1, 801/800, 1]])

In [112]:
sp.linsolve(E1, (x, y))

FiniteSet((-800, 800))

In [113]:
# 2.18 (a)
# Is E1 a linear combinatin of E2 and E3

E1 = sp.Matrix([1, 2, 3])
E2 = sp.Matrix([1, 0, 3])
E3 = sp.Matrix([-1, 1, -3])
RTL([E1, E2, E3])

In [119]:
# Let E4 be an augmented matrix of E1, E2, E3
E4 = sp.Matrix([list(E2), list(E3), list(E1)]).T
E4

Matrix([
[1, -1, 1],
[0,  1, 2],
[3, -3, 3]])

In [121]:
# Find the rref
E4.rref()[0]

Matrix([
[1, 0, 3],
[0, 1, 2],
[0, 0, 0]])

In [124]:
# THis gives values of X and y
# Let E5 be the solution using the rref form of x and y
E5 = 3 * E2 + 2 * E3
E5

Matrix([
[1],
[2],
[3]])

In [129]:
# 2.18 (b)
# Find out if E1 is a linear combination of E2 and E3
E1 = sp.Matrix([2, 3, 4])
E2 = sp.Matrix([1, 0, 3])
E3 = sp.Matrix([-1, 1, -3])

RTL([E1, E2, E3])

In [134]:
# Let E4 be an augmented matrix comprised of E1, E2, and E3
E4 = sp.Matrix([list(E2), list(E3), list(E1)]).T
E4

Matrix([
[1, -1, 2],
[0,  1, 3],
[3, -3, 4]])

In [140]:
# Determine rref 
E4.rref()

(Matrix([
 [1, 0, 0],
 [0, 1, 0],
 [0, 0, 1]]),
 (0, 1, 2))

In [137]:
sp.linsolve(E4, (x, y))

EmptySet

In [ ]:
# No solutions. E